In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
#capturing video
cap = cv.VideoCapture(0)
while cap.isOpened():
    ret , frame = cap.read()
    cv.imshow("Gym Assistant",frame)
    if cv.waitKey(10) & 0xff == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
    

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180:
        angle = 360.0 - angle 
    return angle 
    

In [14]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import sys

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv.VideoCapture(0)
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  

        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            angle = calculate_angle(shoulder, elbow, wrist)

            cv.putText(image, str(angle), tuple(np.multiply(elbow, [640, 480]).astype(int)), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv.LINE_AA)

            if angle > 160:
                stage = "down"
            if angle < 30 and stage == "down":
                stage = "Up"
                counter += 1
                print(counter)
                sys.stdout.flush()  

        except:
            pass

        # ✅ Display counter and stage on the screen  
        cv.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)

        cv.putText(image, 'CNT', (15, 12), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
        cv.putText(image, str(counter), (10, 60), cv.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv.LINE_AA)

        cv.putText(image, 'STAGE', (65, 12), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
        cv.putText(image, stage, (60, 60), cv.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv.LINE_AA)

        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=5),
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=3, circle_radius=5)
        )

        cv.imshow("Gym Assistant", image)  

        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()


1
2
3
4
5
6
